In [6]:
import json, jsonlines, sacrebleu
from torchseq.agents.para_agent import ParaphraseAgent
from torchseq.datasets.json_loader import JsonDataLoader
from torchseq.utils.config import Config

import torch

# Which checkpoint should we load?
path_to_model = '../models/separator-qqp-v1.2/'

# Load the data
with jsonlines.open('../data/qqp-splitforgeneval/test.jsonl') as f:
    rows = [row for row in f]

examples = [{'input': row['sem_input']} for row in rows]


# Change the config to use the custom dataset
with open(path_to_model + "/config.json") as f:
    cfg_dict = json.load(f)
cfg_dict["env"]["data_path"] = "../data/"
cfg_dict["dataset"] = "json"
cfg_dict["json_dataset"] = {
    "path": None,
    "field_map": [
        {"type": "copy", "from": "input", "to": "s2"},
        {"type": "copy", "from": "input", "to": "s1"},
        {"type": "copy", "from": "input", "to": "template"},
    ],
}

# Enable the code predictor
cfg_dict["bottleneck"]["code_predictor"]["infer_codes"] = True

# Create the dataset and model
config = Config(cfg_dict)
data_loader = JsonDataLoader(config, test_samples=examples)
checkpoint_path = path_to_model + "/model/checkpoint.pt"
instance = ParaphraseAgent(config=config, run_id=None, output_path=None, silent=False, verbose=False, training_mode=False)

# Load the checkpoint
instance.load_checkpoint(checkpoint_path)
instance.model.eval()
    
# Finally, run inference
_, _, (pred_output, _, _), _ = instance.inference(data_loader.test_loader)

refs = [q["paras"] for q in rows]
inputs = [[q["sem_input"]] for q in rows]

max_num_refs = max([len(x) for x in refs])
refs_padded = [x + [x[0]] * (max_num_refs - len(x)) for x in refs]

# Calculate uncased iBLEU scores
tgt_bleu = sacrebleu.corpus_bleu(pred_output, list(zip(*refs_padded)), lowercase=True).score
self_bleu = sacrebleu.corpus_bleu(pred_output, list(zip(*inputs)), lowercase=True).score

alpha = 0.7
ibleu = alpha * tgt_bleu - (1 - alpha) * self_bleu

print('Uncased: {:0.2f}/{:0.2f}/{:0.2f}'.format( tgt_bleu, self_bleu, ibleu))

# Calculate cased iBLEU scores
tgt_bleu = sacrebleu.corpus_bleu(pred_output, list(zip(*refs_padded)), lowercase=False).score
self_bleu = sacrebleu.corpus_bleu(pred_output, list(zip(*inputs)), lowercase=False).score

alpha = 0.7
ibleu = alpha * tgt_bleu - (1 - alpha) * self_bleu

print('Cased: {:0.2f}/{:0.2f}/{:0.2f}'.format( tgt_bleu, self_bleu, ibleu))

Validating after 0 epochs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 329/329 [01:51<00:00,  2.96it/s]


Uncased: 22.84/22.93/9.11
Cased: 13.80/14.43/5.33
